# Analyses de contributions

Nous allons réaliser sur notre exemple de béton armé les différentes analyses de contributions vues en cours. Pour rappel, nous nous appuyons sur l'article de van der Meide et al., 2025 (https://link.springer.com/article/10.1007/s11367-025-02487-y#Sec46).

Le procédé étudié a été défini au TD2, il s'agit de la production d'1 m3 de béton armé type A.

Béton | Flux | Procédé choisi dans ecoinvent | Région | Unité | Qté pour 1m3
:---: | :---: | :---: | :---: | :---: | :---:
Béton A | ciment Portland | market for cement, Portland | Europe without Switzerland | kg | 350
Béton A | eau | market for tap water | Europe without Switzerland | kg | 175
Béton A | sable | market for sand | Rest-of-World (RoW) | kg | 800 
Béton A | graviers | market for gravel, crushed | Rest-of-World (RoW) | kg | 1100
Béton A | ferraillage | market for reinforcing steel | Global | kg | 100
Béton A | énergie | diesel, burned in building machine | Global | MJ | 14.4

In [ ]:
import bw2io as bi # ensemble des fonctions et classes pour importer et exporter (input/output)
import bw2data as bd # ... pour gérer les données du projet
import bw2calc as bc # ... pour faire des opérations
import bw2analyzer as ba # ... pour interpréter les résultats
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

bd.projects.set_current('project_ecoinvent_311')
eidb = bd.Database('ecoinvent-3.11-cutoff')
biodb = bd.Database('ecoinvent-3.11-biosphere')
fgdb = bd.Database('betons_armes')

beton_A = fgdb.search("Béton A")[0] # le procédé de béton armé
meth = [m for m in bd.methods if 'EF v3.1' in m[1] and 'no LT' not in m[1]] # la méthode de caractérisation choisie (EF v3.1)

Dans cette séance, nous allons nous intéresser à l'indicateur de potentiel de réchauffement climatique à 100 ans, que l'on isole donc dans une variable.

In [ ]:
gwp100 = [ind for ind in meth if 'GWP100' in str(ind) and 'biogenic' not in str(ind) and 'fossil' not in str(ind) and 'land' not in str(ind)][0]

## AC directe

### Calcul de la matrice de contribution

Afin de réaliser des analyses de contribution directes, on calcule dans ce qui suit des matrices de contribution. Pour cela, on réalise l'ACV du béton A :

In [ ]:
lca = bc.LCA(demand = {beton_A : 1.0},method= gwp100)
lca.lci()
lca.lcia()
s=lca.score
s

In [ ]:
contrib_mat = lca.characterized_inventory # On récupère la matrice d'inventaire caractérisé de brightway, qui est la matrice de contribution
print(contrib_mat)

### AC des flux élémentaires désagrégés

 AC de chaque flux élémentaire de chaque procédé à l'indicateur étudié

On affiche d'abord un tableau de contribution avec un cut-off à 1% :

In [ ]:
# fonction intégrée de brightway qui range les résultats dans un tableau
h_df = lca.to_dataframe(
    matrix_label='characterized_inventory', # on veut les valeurs de la matrice de contribution
    cutoff_mode='fraction', # on va utiliser un cut-off qui sera une fraction du score d'impact
    cutoff=0.01 # on réalise le cut off à 1%
    )

h_df.insert(loc=3,column='relativ_amount',value=h_df['amount']/s)
h_df.insert(loc=4,column='cell_name',value=h_df['row_name'] + "-" + h_df['col_name'])

h_df

#### Exercice

Notre cut-off est-il suffisamment bas pour que notre tableau de flux élémentaires soit à peu près exhaustif sur les contributeurs ? 


#### Correction

On peut sommer les contributions des flux du tableau et les comparer au score total :

In [ ]:
reste = s-sum(h_df['amount'])
print("Le score d'impact total est de",f"{s:.0f}\n",f"La somme des contributions inférieures au cut-off est de {reste:.0f}, soit {reste*100/s :.0f}% du score total")


#### Suite du TD

On peut désormais tracer les contributions des flux élémentaires désagrégés à l'indicateur :

In [ ]:
fig, ax = plt.subplots()
sns.barplot(data=h_df,x = 'cell_name',y = 'relativ_amount',errorbar=None)

plt.xticks(rotation=45, ha='right');


### AC des flux élémentaires agrégés

AC des flux élémentaires agrégés sur l'ensemble des procédés

On stocke la matrice de contribution dans une variable et on regarde à quoi elle ressemble :

In [ ]:
contrib_mat =lca.characterized_inventory # matrice d'inventaire caractérisé ie matrice de contribution
print(contrib_mat)

On peut maintenant sommer les lignes et/ou les colonnes de la matrice pour agréger les contributions par flux élémentaire et/ou procédé.

In [ ]:
contrib_EF = contrib_mat.sum(axis=1) # On fait la somme de chacune des lignes pour avoir la contribution des EF agrégés sur l'ensemble des procédés
print(f"Dimensions de la matrice : {contrib_EF.shape}")
print(f"Somme des éléments de la matrice : {contrib_EF[:,0].sum() : .0f}") # On vérifie que la somme des éléments vaut bien le score total
nodes = [bd.get_node(id = i) for i in lca.dicts.biosphere.keys()] # On crée la liste des noeuds (flux élémentaires) à partir de leurs indices
contrib_EF_dict = {str(nodes[i]):float(contrib_EF[i,0]) for i in range(len(nodes))} # On crée un dictionnaire avec le nom des EF pour clés et la contribution en valeur
print(f"Somme des éléments du dictionnaire : {sum(contrib_EF_dict.values()) : .0f}") # On vérifie que la somme des éléments vaut bien le score total

On peut désormais tracer les contributions, en se fixant un cut-off :

In [ ]:
cutoff = 0.01 # en pourcentage du score d'impact
fig, ax = plt.subplots()
sns.barplot( { k : v for k,v in contrib_EF_dict.items() if v > s*cutoff},errorbar=None) # On fait ici une compréhension de dictionnaire pour ne tracer que les valeurs au dessus du cut-off
plt.xticks(rotation=45, ha='right');

In [ ]:
reste = s-sum({ k : v for k,v in contrib_EF_dict.items() if v > s*cutoff}.values())
print("reste : ",reste,"\n", "reste relatif : ", reste/s) 

On peut se donner un élément de comparaison : 

Regardons la masse de méthane dans notre inventaire et comparons le produit de celle-ci avec un facteur de caractérisation usuel (~29 kgCO2 eq / kg CH4). On veut vérifier que le score d'impact calculé correspond à une caractérisation "faite à la main".

Pour trouver la masse, on fait le même travail que précédemment, mais avec la matrice d'inventaire et non de contribution.

In [ ]:
G =lca.inventory # matrice d'inventaire

G_EF = G.sum(axis=1) # On fait la somme de chacune des lignes pour avoir la contribution à l'inventaire des EF agrégés sur l'ensemble des procédés
nodes = [bd.get_node(id = i) for i in lca.dicts.biosphere.keys()] # On crée la liste des noeuds () à partir de leur indice
G_EF = {str(nodes[i]) : float(G_EF[i,0]) for i in range(len(nodes))} # On crée un dictionnaire avec le nom des EF pour clés et la contribution en valeur
G_EF = { k : v for k,v in G_EF.items() if v!=0} # On supprime les valeurs nulles

On affiche désormais toutes les quantités de méthane de notre inventaire :

In [ ]:
methanes = {k : G_EF[k] for k in G_EF.keys() if 'Methane' in str(k)}
print(methanes)
contrib_methanes = {k : contrib_EF_dict[k] for k in contrib_EF_dict.keys() if 'Methane' in str(k)}
methanes

In [ ]:
masse_methane = sum(methanes.values())
CF_usuel = 29
resultat_coindetable = masse_methane*CF_usuel
resultat_brightway = sum(contrib_methanes.values())
print('coin de table : ',resultat_coindetable,'\n','brightway : ',resultat_brightway)

Le résultat attendu est proche de celui obtenu. Vérifions le facteur de caractérisation utilisé par Brightway pour notre méthode.

In [ ]:
methane_ids = {node : node.id for node in biodb if node['name'] == 'Methane, fossil' } # On regarde l'indice des noeuds correspondant au méthane
methane_ids


In [ ]:
C = lca.characterization_matrix # On regarde dans la matrice de caractérisation la valeur qui nous intéresse
C[lca.dicts.biosphere[list(methane_ids.values())[3]],lca.dicts.biosphere[list(methane_ids.values())[3]]]


### AC des procédés

Pour trouver la contribution directe de chacun des procédés, la démarche est exactement la même, mais en agrégeant suivant les colonnes.

In [ ]:
contrib_process = contrib_mat.sum(axis=0)
print(f"Dimensions de la matrice : {contrib_process.shape}")
print(f"Somme des éléments de la matrice : {contrib_process[0,:].sum() : .0f}") # On vérifie que la somme des éléments vaut bien le score total
nodes = [bd.get_node(id = i) for i in lca.dicts.activity.keys()] # On crée la liste des noeuds (ici des procédés) à partir de leurs indices
contrib_process_dict = {nodes[i]:float(contrib_process[0,i]) for i in range(len(nodes))} # On crée un dictionnaire avec le nom des procédés pour clés et la contribution en valeur
print(f"Somme des éléments du dictionnaire : {sum(contrib_EF_dict.values()) : .0f}") # On vérifie que la somme des éléments vaut bien le score total

On peut désormais tracer les contributions, en se fixant un cut-off :

In [ ]:
cutoff = 0.01 # en pourcentage du score d'impact
fig, ax = plt.subplots()
sns.barplot( { k : v/s for k,v in contrib_process_dict.items() if v > s*cutoff},errorbar=None) # On fait ici une compréhension de dictionnaire pour ne tracer que les valeurs au dessus du cut-off
plt.xticks(rotation=45, ha='right');

### AC de groupes de procédés

On peut grouper les procédés et évaluer la contribution directe de ces groupes aux indicateurs d'impacts. Le crtière pour les grouper est libre, il est courant de regrouper les procédés par étape du cycle de vie, ou par secteur de l'économie.

On commence par reprendre le dicitonnaire des contributions par procédés, et on agrège selon le critère du secteur économique.

In [ ]:
filtered_contrib_process_dict = { k : v for k,v in contrib_process_dict.items() if 'classifications' in list(k.keys())} # On ne garde que les activités qui ont une classification (deux activités posent problèmes seulement)
sectors = set([act['classifications'][0][1] for act in list(filtered_contrib_process_dict.keys())]) # On détermine l'ensemble des secteurs, avec une fois chaque secteur
contrib_sectors = {sector : sum([i[1] for i in filtered_contrib_process_dict.items() if i[0]['classifications'][0][1]==sector]) for sector in sectors} # Pour chaque secteur, on somme la liste des contributions qui en font partie

On vérifie que la somme des contributions vaut le score total :

In [ ]:
print(f"Somme des éléments du dictionnaire : {sum(contrib_sectors.values()) : .0f}")

On peut désormais tracer les contributions, en se fixant un cut-off :

In [ ]:
cutoff = 0.01 # en pourcentage du score d'impact

fig, ax = plt.subplots()
sns.barplot( { k : v/s for k,v in contrib_sectors.items() if v > s*cutoff},errorbar=None) # On fait ici une compréhension de dictionnaire pour ne tracer que les valeurs au dessus du cut-off
plt.xticks(rotation=45, ha='right');

Nous nous sommes ici servis du fait que les secteurs de l'économie sont une donnée présente dans les procédés ecoinvent. Si vous souhaitez grouper les activités que vous créez par étapes du cycle de vie, pensez à l'indiquer en métadonnées, dans un champ que vous pouvez créer, ou bien faite des macro-procédés par étape de cycle de vie.

## AC indirecte

### AC des procédés de premier niveau

Pour avoir les contributions des procédés de premier niveau, on réalise successivement l'ACV de chacun d'eux. On récupère donc les activités avec lesquelles notre béton a des échanges (flux intermédiaires).

In [ ]:
first_tier_processes ={p.input : p.amount for p in list(beton_A.technosphere())} # On récupère les identifiants des procédés et la valeurs de flux intermédiaires associés

In [ ]:
contrib_first_processes = {}
for p,a in first_tier_processes.items() :
    lca = bc.LCA(demand={p : a},method=gwp100)
    lca.lci()
    lca.lcia()
    contrib_first_processes[p['name']] = lca.score

In [ ]:
cutoff = 0.005 # en pourcentage du score d'impact

fig, ax = plt.subplots()
sns.barplot( { k : v/s for k,v in contrib_first_processes.items() if v > s*cutoff},errorbar=None) # On fait ici une compréhension de dictionnaire pour ne tracer que les valeurs au dessus du cut-off
plt.xticks(rotation=45, ha='right');

Et si on changeait le cut-off ?

On peut également tracer un diagramme en barres empilées :

In [ ]:
df_first_processes = pd.DataFrame(contrib_first_processes,index=[str(gwp100[-1])])
df_first_processes.plot(kind='bar', stacked=True)
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.xticks(rotation=0, ha='center');